In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
#specify.master("yarn")

sc = spark.sparkContext

In [3]:
# Here is the code for making connectivity to hive metastore from pyspark

spark = spark.builder.master('yarn').config('spark.sql.warehouse.dir', '/user/hive/warehouse').config('hive.metastore.uris', 'thrift://localhost:9083').enableHiveSupport().getOrCreate()
spark = spark.builder.master('yarn').enableHiveSupport().getOrCreate()
spark.sql("show databases").show()
spark.sql("use project").show()
spark.sql("show tables").show()
print(" - ")
spark.table('cars').show()
print(spark.catalog.listTables())
print(spark.conf.get('spark.sql.warehouse.dir'))

+------------+
|databaseName|
+------------+
|     default|
|     project|
+------------+

++
||
++
++

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| project|     cars|      false|
+--------+---------+-----------+

 - 
+-----+----+-------+----------------+-----+-----------------+-----+------------+
|price|year|mileage|            city|state|              vin| make|       model|
+-----+----+-------+----------------+-----+-----------------+-----+------------+
| null|null|   null|            City|State|              Vin| Make|       Model|
| 8995|2014|  35725|         El Paso|   TX|19VDE2E53EE000083|Acura|  ILX6-Speed|
|10888|2013|  19606|Long Island City|   NY|19VDE1F52DE012636|Acura|  ILX5-Speed|
| 8995|2013|  48851|         El Paso|   TX|19VDE2E52DE000025|Acura|  ILX6-Speed|
|10999|2014|  39922|         Windsor|   CO|19VDE1F71EE003817|Acura|  ILX5-Speed|
|14799|2016|  22142|          Lindon|   UT|19UDE2F32GA001284|Acura|ILXAutomati

In [4]:
df=spark.read.table("cars")

In [5]:
df.columns

['price', 'year', 'mileage', 'city', 'state', 'vin', 'make', 'model']

In [6]:
df.show()

+-----+----+-------+----------------+-----+-----------------+-----+------------+
|price|year|mileage|            city|state|              vin| make|       model|
+-----+----+-------+----------------+-----+-----------------+-----+------------+
| null|null|   null|            City|State|              Vin| Make|       Model|
| 8995|2014|  35725|         El Paso|   TX|19VDE2E53EE000083|Acura|  ILX6-Speed|
|10888|2013|  19606|Long Island City|   NY|19VDE1F52DE012636|Acura|  ILX5-Speed|
| 8995|2013|  48851|         El Paso|   TX|19VDE2E52DE000025|Acura|  ILX6-Speed|
|10999|2014|  39922|         Windsor|   CO|19VDE1F71EE003817|Acura|  ILX5-Speed|
|14799|2016|  22142|          Lindon|   UT|19UDE2F32GA001284|Acura|ILXAutomatic|
| 7989|2012| 105246|           Miami|   FL|JH4CU2F83CC019895|Acura|TSXAutomatic|
|14490|2014|  34032|       Greatneck|   NY|JH4CU2F84EC002686|Acura|  TSXSpecial|
|13995|2013|  32384|     West Jordan|   UT|JH4CU2F64DC006203|Acura|  TSX5-Speed|
|10495|2013|  57596|       W

In [7]:
cars_raw = df.sample(False, 0.65, seed=777)

In [8]:
df=cars_raw.drop("City","State","Vin")

In [9]:
df=df.dropna()

In [10]:
df.show()

+-----+----+-------+-----+------------+
|price|year|mileage| make|       model|
+-----+----+-------+-----+------------+
| 8995|2013|  48851|Acura|  ILX6-Speed|
|13995|2013|  32384|Acura|  TSX5-Speed|
|10495|2013|  57596|Acura|  ILX6-Speed|
|12921|2012|  58550|Acura|TSXAutomatic|
|17628|2015|  13797|Acura|  ILX5-Speed|
|13999|2013|  35035|Acura|  TSX5-Speed|
|14995|2014|  23454|Acura|  ILX5-Speed|
|14990|2015|  23603|Acura|  ILX5-Speed|
|14590|2010|  19250|Acura|      TSX4dr|
| 9500|2011|  68289|Acura|      TSX4dr|
|16994|2015|  23946|Acura|  ILX5-Speed|
|15499|2014|  27171|Acura|  TSX5-Speed|
|13499|2014|  35037|Acura|  ILX5-Speed|
|14999|2014|  17669|Acura|  ILX5-Speed|
|14500|2010|  25926|Acura|      TSX4dr|
|16000|2015|  30881|Acura|  ILX5-Speed|
|17419|2015|  15390|Acura|  ILX5-Speed|
|14999|2015|  27333|Acura|  ILX5-Speed|
|14999|2015|  28326|Acura|  ILX5-Speed|
|17000|2015|  24671|Acura|  ILX5-Speed|
+-----+----+-------+-----+------------+
only showing top 20 rows



In [11]:
df.dropDuplicates().show()

+-----+----+-------+-----+------------+
|price|year|mileage| make|       model|
+-----+----+-------+-----+------------+
|13998|2010|  41652|Acura|      TSX4dr|
|16750|2015|  57483|Acura|  ILX5-Speed|
|16517|2015|  48918|Acura|  ILX5-Speed|
|19968|2016|  31019|Acura|ILXAutomatic|
|17384|2013|  35257|Acura|      TLBase|
|14539|2012|  59046|Acura|       TL2WD|
| 7995|2008|  78692|Acura|      RDX4WD|
| 4988|2000| 109624|Acura| TLAutomatic|
|22500|2014|  35341|Acura|       TL2WD|
|15000|2011|  80442|Acura|      RDXFWD|
|20190|2015|  31235|Acura|      TLXFWD|
|23499|2014|  48194|Acura|    TLSH-AWD|
|21800|2015|  32649|Acura|      TLXFWD|
|27999|2016|  10398|Acura|      TLXFWD|
|27749|2016|  15429|Acura|      TLXFWD|
|25656|2015|  29778|Acura|      TLXFWD|
|16734|2012|  77424|Acura|       TL2WD|
|33990|2016|   7205|Acura|      RDXFWD|
|24420|2013|  56175|Acura|     MDXwith|
| 4499|2003| 184707|Acura| TLAutomatic|
+-----+----+-------+-----+------------+
only showing top 20 rows



In [12]:
from pyspark.ml.linalg import Vector 
from pyspark.ml.feature import VectorAssembler 

In [13]:
df.columns

['price', 'year', 'mileage', 'make', 'model']

In [16]:
# Preprocessing: StringIndexer for categorical labels

from pyspark.ml.feature import StringIndexer


stringIndexer1  = StringIndexer(inputCol="model", outputCol="model_label")


model=stringIndexer1.fit(df)
indexed1=model.transform(df)

stringIndexer2  = StringIndexer(inputCol="make", outputCol="make_label")
model2=stringIndexer2.fit(indexed1)
indexed2=model2.transform(indexed1)

In [17]:
from pyspark.ml.linalg import Vector 
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['year', 'mileage', 'make_label', 'model_label'], outputCol="features")



In [18]:
indexed2.show()

+-----+----+-------+-----+------------+-----------+----------+
|price|year|mileage| make|       model|model_label|make_label|
+-----+----+-------+-----+------------+-----------+----------+
| 8995|2013|  48851|Acura|  ILX6-Speed|     1611.0|      22.0|
|13995|2013|  32384|Acura|  TSX5-Speed|      726.0|      22.0|
|10495|2013|  57596|Acura|  ILX6-Speed|     1611.0|      22.0|
|12921|2012|  58550|Acura|TSXAutomatic|      646.0|      22.0|
|17628|2015|  13797|Acura|  ILX5-Speed|      300.0|      22.0|
|13999|2013|  35035|Acura|  TSX5-Speed|      726.0|      22.0|
|14995|2014|  23454|Acura|  ILX5-Speed|      300.0|      22.0|
|14990|2015|  23603|Acura|  ILX5-Speed|      300.0|      22.0|
|14590|2010|  19250|Acura|      TSX4dr|      431.0|      22.0|
| 9500|2011|  68289|Acura|      TSX4dr|      431.0|      22.0|
|16994|2015|  23946|Acura|  ILX5-Speed|      300.0|      22.0|
|15499|2014|  27171|Acura|  TSX5-Speed|      726.0|      22.0|
|13499|2014|  35037|Acura|  ILX5-Speed|      300.0|    

In [19]:
output=assembler.transform(indexed2)

In [20]:
output.show(truncate=False)

+-----+----+-------+-----+------------+-----------+----------+----------------------------+
|price|year|mileage|make |model       |model_label|make_label|features                    |
+-----+----+-------+-----+------------+-----------+----------+----------------------------+
|8995 |2013|48851  |Acura|ILX6-Speed  |1611.0     |22.0      |[2013.0,48851.0,22.0,1611.0]|
|13995|2013|32384  |Acura|TSX5-Speed  |726.0      |22.0      |[2013.0,32384.0,22.0,726.0] |
|10495|2013|57596  |Acura|ILX6-Speed  |1611.0     |22.0      |[2013.0,57596.0,22.0,1611.0]|
|12921|2012|58550  |Acura|TSXAutomatic|646.0      |22.0      |[2012.0,58550.0,22.0,646.0] |
|17628|2015|13797  |Acura|ILX5-Speed  |300.0      |22.0      |[2015.0,13797.0,22.0,300.0] |
|13999|2013|35035  |Acura|TSX5-Speed  |726.0      |22.0      |[2013.0,35035.0,22.0,726.0] |
|14995|2014|23454  |Acura|ILX5-Speed  |300.0      |22.0      |[2014.0,23454.0,22.0,300.0] |
|14990|2015|23603  |Acura|ILX5-Speed  |300.0      |22.0      |[2015.0,23603.0,22

In [21]:
train_data, test_data = output.randomSplit([0.8, 0.2], seed=123)

In [22]:
# Create a RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressor
rf_regressor = RandomForestRegressor(featuresCol="features", labelCol="price", numTrees=150, maxBins=3000)


In [23]:
rf_model = rf_regressor.fit(train_data)

In [24]:
predictions = rf_model.transform(test_data)

In [25]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 6353.793822417411


In [26]:
# R-squared (R2)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print("R-squared (R2):", r2)

R-squared (R2): 0.7792687452839657


# transformations

In [29]:
df=spark.read.table("cars")

In [41]:
df=df.dropna()

In [42]:
df1=df.orderBy(df.city.asc(),df.price.asc())

In [44]:
df1.show() #cities and price sorted in ascending order

+-----+----+-------+----------+-----+-----------------+-------------+---------------+
|price|year|mileage|      city|state|              vin|         make|          model|
+-----+----+-------+----------+-----+-----------------+-------------+---------------+
|10339|2011| 102333|     AKRON|   OH|WDDGF8BB2BR150111|Mercedes-Benz|     C-Class4dr|
|15889|2011|  92812|     AKRON|   OH|4JGBB8GB0BA681651|Mercedes-Benz|  M-Class4MATIC|
|16999|2008|  39493|     AKRON|   OH|4JGBB22EX8A421443|Mercedes-Benz|     M-Class4WD|
|17889|2011|  72832|     AKRON|   OH|WDDHF8HB4BA501126|Mercedes-Benz|     E-Class4dr|
|23876|2013|  39961|     AKRON|   OH|WDCGG8JB1DG016238|Mercedes-Benz|GLK-ClassGLK350|
|25229|2014|  66378|     AKRON|   OH|WDCGG8JB4EG326238|Mercedes-Benz|GLK-ClassGLK350|
|26449|2004|  25403|     AKRON|   OH|WDBSK75F24F071548|Mercedes-Benz|  SL-ClassSL500|
|33959|2014|  35461|     AKRON|   OH|4JGDA5HB1EA353653|Mercedes-Benz|   M-ClassML350|
|35889|2015|  29610|     AKRON|   OH|WDCGG8JB8FG378795

In [56]:
df1.coalesce(1).write.format("csv").mode('overwrite').save("/user/talentum/tfm1")

In [43]:
df2=df.groupBy("year").count()

In [47]:
df2.sort(df.year.desc()).show() #yearly count of cars sold in descending order

+----+------+
|year| count|
+----+------+
|2018|   922|
|2017| 91608|
|2016|132136|
|2015|157516|
|2014|162432|
|2013| 74701|
|2012| 49764|
|2011| 39768|
|2010| 27539|
|2009| 19061|
|2008| 24713|
|2007| 21171|
|2006| 15079|
|2005| 11005|
|2004|  8117|
|2003|  5649|
|2002|  3800|
|2001|  2584|
|2000|  1933|
|1999|  1254|
+----+------+
only showing top 20 rows



In [57]:
df2.coalesce(1).write.format("csv").mode('overwrite').save("/user/talentum/tfm2")

In [50]:
df3=df.groupBy("make").count()
df3.show() 

+-----------+------+
|       make| count|
+-----------+------+
| Volkswagen| 23249|
| Oldsmobile|   122|
|         AM|    19|
|      Lexus| 20641|
|     Jaguar|  2200|
|     Saturn|   963|
|       FIAT|  1782|
|   Maserati|  1047|
|Rolls-Royce|    92|
|      Scion|  3043|
|       Jeep| 40373|
| Mitsubishi|  4080|
|        Kia| 28636|
|  Chevrolet|102268|
|      Volvo|  5106|
|    Hyundai| 35837|
|       Saab|   260|
|      Honda| 50193|
|   INFINITI| 12258|
|       MINI|  4375|
+-----------+------+
only showing top 20 rows



In [58]:
df3.coalesce(1).write.format("csv").mode('overwrite').save("/user/talentum/tfm3")

In [62]:
df4=df.filter(df.city=='El Paso').groupBy("make").count()

In [63]:
df4.show()

+----------+-----+
|      make|count|
+----------+-----+
|Volkswagen|  100|
|     Lexus|   34|
|    Jaguar|    5|
|    Saturn|    5|
|      FIAT|   12|
|  Maserati|    1|
|     Scion|   12|
|      Jeep|  193|
|Mitsubishi|   15|
|       Kia|   77|
| Chevrolet|  475|
|     Volvo|    2|
|   Hyundai|  138|
|     Honda|  184|
|  INFINITI|   31|
|      MINI|   19|
|      Audi|   18|
|       Ram|   66|
|  Cadillac|   56|
|   Pontiac|    6|
+----------+-----+
only showing top 20 rows



In [64]:
df4.coalesce(1).write.format("csv").mode('overwrite').save("/user/talentum/tfm4")